**CLEANING Rates_of_Cases_or_Deaths.csv** (First dataset)

In [1]:
import pandas as pd
import shutil
import re
import numpy as np

In [2]:
raw = pd.read_csv("Data/raw/Rates_of_Cases_or_Deaths.csv")
print(raw)

     outcome     month  MMWR week     Age group Vaccine product  \
0       case  APR 2021     202114         12-17       all_types   
1       case  APR 2021     202114         18-29       all_types   
2       case  APR 2021     202114         30-49       all_types   
3       case  APR 2021     202114         50-64       all_types   
4       case  APR 2021     202114         65-79       all_types   
...      ...       ...        ...           ...             ...   
1586   death  AUG 2022     202235           80+       all_types   
1587   death  AUG 2022     202235  all_ages_adj       all_types   
1588   death  AUG 2022     202235  all_ages_adj         Janssen   
1589   death  AUG 2022     202235  all_ages_adj         Moderna   
1590   death  AUG 2022     202235  all_ages_adj          Pfizer   

      Vaccinated with outcome  Fully vaccinated population  \
0                           8                 3.688700e+04   
1                         674                 2.543093e+06   
2        

***DROPPING UNCESSARY COLUMNS***

For this analysis, the columns needed in the first dataset (Rates_of_Cases_or_Deaths.csv) are outcome', 'month', 'Age group', 'Vaccinated with outcome', and Unvaccinated with outcome. The other columns are to be dropped.

In [3]:
raw.columns

Index(['outcome', 'month', 'MMWR week', 'Age group', 'Vaccine product',
       'Vaccinated with outcome', 'Fully vaccinated population',
       'Unvaccinated with outcome', 'Unvaccinated population', 'Crude vax IR',
       'Crude unvax IR', 'Crude IRR', 'Age adjusted vax IR',
       'Age adjusted unvax IR', 'Age adjusted IRR', 'Continuity correction'],
      dtype='object')

In [6]:
clean = raw.drop(["MMWR week", "Vaccine product", "Fully vaccinated population", "Unvaccinated population", "Crude vax IR",
       "Crude unvax IR", "Crude IRR","Age adjusted vax IR", "Age adjusted unvax IR", "Age adjusted IRR","Continuity correction"], axis=1)
print(clean)

     outcome     month     Age group  Vaccinated with outcome  \
0       case  APR 2021         12-17                        8   
1       case  APR 2021         18-29                      674   
2       case  APR 2021         30-49                     1847   
3       case  APR 2021         50-64                     1558   
4       case  APR 2021         65-79                     1672   
...      ...       ...           ...                      ...   
1586   death  AUG 2022           80+                      328   
1587   death  AUG 2022  all_ages_adj                      603   
1588   death  AUG 2022  all_ages_adj                       33   
1589   death  AUG 2022  all_ages_adj                      286   
1590   death  AUG 2022  all_ages_adj                      280   

      Unvaccinated with outcome  
0                         30785  
1                         76736  
2                         98436  
3                         50324  
4                         14880  
...            

In [7]:
# Save clean to csv file
clean.to_csv("clean_data_1.csv", index=False)

In [14]:
# Load the clean csv file to Data/clean/ folder
source = "clean_data_1.csv"
destination = "Data/clean/"
shutil.move(source, destination)

'Data/clean/clean_data_small.csv'

***HANDLING MISSING AND INVALID DATA***

In [8]:
# Read the csv file
clean = pd.read_csv("Data/clean/clean_data_1.csv")

In [9]:
# Check the null values
clean.isnull().sum()

outcome                      0
month                        0
Age group                    0
Vaccinated with outcome      0
Unvaccinated with outcome    0
dtype: int64

*There is 0 null values in each column.*

In [10]:
# Define a function to check for invalid entries in the "outcome" column

def is_valid_outcome(value):
    if pd.isna(value): 
        return False
    return bool(re.match(r"^(case|death)$", str(value).strip()))  # Normalize value and check match

# Filter invalid entries
invalid_outcome = clean[~clean['outcome'].apply(is_valid_outcome)]

# Count invalid entries
print(f"Number of invalid entries: {len(invalid_outcome)}")

# Verify the cleaning process
print("Unique values after cleaning:", clean['outcome'].unique())

Number of invalid entries: 0
Unique values after cleaning: ['case' 'death']


In [12]:
# Check for invalid entries in the month column.

# Define a valid pattern for month-year
def is_valid_month(value):
    if pd.isna(value):
        return False
    return bool(re.match(r"^(Jan|Feb|Mar|Apr|May|June|July|Aug|Sept|Oct|Nov|Dec)(2021|2022)$", str(value).strip()))

# Identify invalid entries
invalid_month = clean[~clean['month'].apply(is_valid_month)]

# Display invalid entries
print("Invalid entries in 'month' column:")
print(invalid_month)

# Count invalid entries
print(f"Number of invalid entries: {len(invalid_month)}")

# Verify the cleaning process
print("Unique values after cleaning:", clean['month'].unique())

Invalid entries in 'month' column:
     outcome     month     Age group  Vaccinated with outcome  \
0       case  APR 2021         12-17                        8   
1       case  APR 2021         18-29                      674   
2       case  APR 2021         30-49                     1847   
3       case  APR 2021         50-64                     1558   
4       case  APR 2021         65-79                     1672   
...      ...       ...           ...                      ...   
1586   death  AUG 2022           80+                      328   
1587   death  AUG 2022  all_ages_adj                      603   
1588   death  AUG 2022  all_ages_adj                       33   
1589   death  AUG 2022  all_ages_adj                      286   
1590   death  AUG 2022  all_ages_adj                      280   

      Unvaccinated with outcome  
0                         30785  
1                         76736  
2                         98436  
3                         50324  
4             

In [13]:
# checking for invalid values in age group column.

unique_age_groups = clean['Age group'].unique()

# valid age group intervals 
expected_age_groups = unique_age_groups = [
    "12-17", "18-29", "30-49", "50-64", "65-79", "80+", "5-11", "all_ages_adj"
]

# Check if all values in the 'Age group' column are valid
unexpected_age_groups = clean[~clean['Age group'].isin(expected_age_groups)]

print("Unexpected Age group values:")
print(unexpected_age_groups['Age group'].unique())

print(f"Number of unexpected age groups: {len(unexpected_age_groups)}")
print(unique_age_groups)

Unexpected Age group values:
[]
Number of unexpected age groups: 0
['12-17', '18-29', '30-49', '50-64', '65-79', '80+', '5-11', 'all_ages_adj']


In [14]:
clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1591 entries, 0 to 1590
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   outcome                    1591 non-null   object
 1   month                      1591 non-null   object
 2   Age group                  1591 non-null   object
 3   Vaccinated with outcome    1591 non-null   int64 
 4   Unvaccinated with outcome  1591 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 62.3+ KB


In [ ]:
# List of months to fill missing values

months = ['APR 2021', 'MAY 2021', 'JUN 2021', 'JUL 2021', 'AUG 2021', 
          'SEP 2021', 'OCT 2021', 'NOV 2021', 'DEC 2021', 'JAN 2022', 
          'FEB 2022', 'MAR 2022', 'APR 2022', 'MAY 2022', 'JUN 2022', 
          'JUL 2022', 'AUG 2022', 'SEP 2022']

# Replace NaN values sequentially with the provided list.        
nan_indices = clean['month'][clean['month'].isna()].index
for i, idx in enumerate(nan_indices):
    clean.loc[idx, 'month'] = months[i % len(months)]
print(clean['month'])

Explanation of the Code
nan_indices: Finds the indices of rows in the month column with NaN values.
Loop:
i helps track the position in the list of nan_indices.
idx provides the index of the specific row with a NaN.
months[i % len(months)] cycles through the months list repeatedly using modulo arithmetic (%).
The code assigns a value from months to the NaN positions in a cyclical manner.
Result: The previously NaN values in the month column are filled with values from the months list in a repeating pattern.


In [17]:
clean.duplicated().sum()

np.int64(38)

In [18]:
duplicate_count = int(clean.duplicated().sum())
duplicate_count

38

In [19]:
# Identify duplicate rows
duplicates = clean[clean.duplicated()]
print(f"Number of duplicate rows: {duplicates.shape[0]}")

Number of duplicate rows: 38


In [20]:
# Create clean_back to avoid changing clean in case the duplicates come from filling in the month column
clean_backup = clean.copy()

In [21]:
# Add a column to flag duplicates
clean_backup['is_duplicate'] = clean_backup.duplicated()

**CLEANING County_Level_of_community_Transmission.csv (second dataset)**

In [31]:
raw = pd.read_csv("Data/raw/County_Level_of_community_Transmission.csv")
print(raw)

           state_name        county_name  fips_code        date  \
0               Texas     Jackson County      48239  01/27/2020   
1            Nebraska     Buffalo County      31019  01/27/2020   
2            Illinois       Coles County      17029  01/27/2020   
3                Iowa         Ida County      19093  01/27/2020   
4           Tennessee        Knox County      47093  02/05/2020   
...               ...                ...        ...         ...   
3225217     Louisiana  Assumption Parish      22007  10/18/2022   
3225218  South Dakota        Hyde County      46069  10/18/2022   
3225219         Texas      Loving County      48301  10/18/2022   
3225220      Virginia     Lynchburg City      51680  10/18/2022   
3225221       Wyoming    Converse County      56009  10/18/2022   

        cases_per_100K_7_day_count_change  \
0                                   0.000   
1                                   0.000   
2                                   0.000   
3              

***DROPPING UNCESSARY COLUMNS***

*For the second dataset (County_Level_of_community_Transmission.csv) important columns are state_name', 'date', 'community_transmission_level'.* 


In [32]:
raw.columns

Index(['state_name', 'county_name', 'fips_code', 'date',
       'cases_per_100K_7_day_count_change',
       'percent_test_results_reported_positive_last_7_days',
       'community_transmission_level'],
      dtype='object')

In [33]:
clean = raw.drop(['county_name', 'fips_code',
       'cases_per_100K_7_day_count_change',
       'percent_test_results_reported_positive_last_7_days'], axis=1)
print(clean)

           state_name        date community_transmission_level
0               Texas  01/27/2020                          NaN
1            Nebraska  01/27/2020                          NaN
2            Illinois  01/27/2020                          NaN
3                Iowa  01/27/2020                          NaN
4           Tennessee  02/05/2020                          low
...               ...         ...                          ...
3225217     Louisiana  10/18/2022                          low
3225218  South Dakota  10/18/2022                          low
3225219         Texas  10/18/2022                         high
3225220      Virginia  10/18/2022                  substantial
3225221       Wyoming  10/18/2022                     moderate

[3225222 rows x 3 columns]


In [34]:
clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3225222 entries, 0 to 3225221
Data columns (total 3 columns):
 #   Column                        Dtype 
---  ------                        ----- 
 0   state_name                    object
 1   date                          object
 2   community_transmission_level  object
dtypes: object(3)
memory usage: 73.8+ MB


***MISSING AND INVALIDE VALUES***

In [35]:
clean.isnull().sum()

state_name                       2662
date                                0
community_transmission_level    22661
dtype: int64

In [36]:
# % of missing values in each column = number of missing values/total values * 100
clean.isnull().sum()/len(clean) * 100

state_name                      0.082537
date                            0.000000
community_transmission_level    0.702618
dtype: float64

In [38]:
# Define a function to check for valid entries in the "state_name" column

def is_valid_state_name(value):
    if pd.isna(value):  # Check for missing values
        return False
    return bool(re.match(r"^(value)$", str(value).strip()))

In [40]:
  # Normalize value and check match

# Filter invalid entries
invalid_state_name = clean[~clean['state_name'].apply(is_valid_state_name)]

In [41]:
# Count invalid entries
print(f"Number of invalid entries: {len(invalid_state_name)}")

# Verify the cleaning process
print("Unique values after cleaning:", clean['state_name'].unique())

Number of invalid entries: 3225222
Unique values after cleaning: ['Texas' 'Nebraska' 'Illinois' 'Iowa' 'Tennessee' 'Wisconsin' 'Missouri'
 'New York' 'Alabama' 'Ohio' 'Utah' 'Georgia' 'Minnesota' 'Alaska'
 'New Jersey' 'Colorado' 'South Dakota' 'West Virginia' 'California'
 'Michigan' 'Kansas' 'Nevada' 'Montana' 'Kentucky' 'Oregon' 'Indiana'
 'Mississippi' 'Virginia' 'Idaho' 'South Carolina' 'North Carolina'
 'Oklahoma' 'Florida' 'Hawaii' 'North Dakota' 'Arkansas' 'Maine'
 'Pennsylvania' 'Vermont' 'Wyoming' 'Puerto Rico' 'Arizona' 'New Mexico'
 'Washington' 'Louisiana' 'Connecticut' 'Maryland' nan
 'District of Columbia' 'Delaware' 'Massachusetts' 'Rhode Island'
 'New Hampshire']


In [44]:
# Replace the nan by the mode_value
mode_value = clean['state_name'].mode().iloc[0]
clean['state_name'] = clean['state_name'].fillna(mode_value)
print("Unique values after cleaning:", clean['state_name'].unique())

Unique values after cleaning: ['Texas' 'Nebraska' 'Illinois' 'Iowa' 'Tennessee' 'Wisconsin' 'Missouri'
 'New York' 'Alabama' 'Ohio' 'Utah' 'Georgia' 'Minnesota' 'Alaska'
 'New Jersey' 'Colorado' 'South Dakota' 'West Virginia' 'California'
 'Michigan' 'Kansas' 'Nevada' 'Montana' 'Kentucky' 'Oregon' 'Indiana'
 'Mississippi' 'Virginia' 'Idaho' 'South Carolina' 'North Carolina'
 'Oklahoma' 'Florida' 'Hawaii' 'North Dakota' 'Arkansas' 'Maine'
 'Pennsylvania' 'Vermont' 'Wyoming' 'Puerto Rico' 'Arizona' 'New Mexico'
 'Washington' 'Louisiana' 'Connecticut' 'Maryland' 'District of Columbia'
 'Delaware' 'Massachusetts' 'Rhode Island' 'New Hampshire']


In [43]:
# Define a function to check for valid entries in the "community_transmission_level" column

def is_valid_community_transmission_level(value):
    if pd.isna(value):  # Check for missing values
        return False
    return bool(re.match(r"^(high|low|moderate|substantial)$", str(value).strip()))  # Normalize value and check match

# Filter invalid entries
invalid_community_transmission_level = clean[~clean['community_transmission_level'].apply(is_valid_community_transmission_level)]

# Count invalid entries
print(f"Number of invalid entries: {len(invalid_community_transmission_level)}")

# Verify the cleaning process
print("Unique values after cleaning:", clean['community_transmission_level'].unique())

Number of invalid entries: 22661
Unique values after cleaning: [nan 'low' 'high' 'moderate' 'substantial']


In [45]:
# Replace 'nan' by mode_value
mode_value = clean['community_transmission_level'].mode().iloc[0]
clean['community_transmission_level'] = clean['community_transmission_level'].fillna(mode_value)
print("Unique values after cleaning:", clean['community_transmission_level'].unique())

Unique values after cleaning: ['high' 'low' 'moderate' 'substantial']


In [47]:
clean.to_csv('clean_data_2.csv', index=False)

In [31]:
source = "clean_data_2.csv"
destination = "Data/clean/"
shutil.move(source, destination)

'Data/clean/clean_data_2.csv'

***CHECKING DUPLICATES***

In [48]:
clean.duplicated().sum()
duplicat_count = int(clean.duplicated().sum())
duplicat_count

3084721

In [50]:
# Create clean_backup to avoid changing clean in case the duplicates come from filling in the columns
clean_backup = clean.copy()
# Add a column to flag duplicates
clean_backup['is_duplicate'] = clean_backup.duplicated()